In [1]:
hadoop_versions = [
         "2.9.2", "2.9.1", "2.9.0","2.8.5", "2.8.4", "2.8.3","2.8.2", "2.8.1","2.8.0",
        "2.7.7","2.7.6","2.7.5","2.7.3","2.7.2","2.7.1","2.7.0","2.6.5", "2.6.4","2.6.3"
        ,"2.6.2","2.6.1","2.6.0","2.5.2","2.5.1","2.5.0","2.4.1","2.4.0","2.3.0"
        ,"3.3.0", "3.3.1", "3.3.2", "3.3.3", "3.3.4", "3.3.5","3.3.6"
       ,"3.2.0", "3.3.1", "3.2.2", "3.2.3", "3.2.4"
       ,"3.1.0","3.1.1","3.1.2","3.1.3","3.1.4"
       ,"3.0.0","3.0.1","3.0.2","3.0.3"
    ]


neededColumnsClass =['cbo','cboModified','fanin','fanout','wmc','dit','noc','lcom',
                     'lcom*','tcc','lcc', 'loc'
               ]

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jan  8 01:47:25 2024

@author: ayferuckan
"""
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

hadoop_versions = [
         "2.9.2", "2.9.1", "2.9.0","2.8.5", "2.8.4", "2.8.3","2.8.2", "2.8.1","2.8.0",
        "2.7.7","2.7.6","2.7.5","2.7.3","2.7.2","2.7.1","2.7.0","2.6.5", "2.6.4","2.6.3"
        ,"2.6.2","2.6.1","2.6.0","2.5.2","2.5.1","2.5.0","2.4.1","2.4.0","2.3.0"
        ,"3.3.0", "3.3.1", "3.3.2", "3.3.3", "3.3.4", "3.3.5","3.3.6"
       ,"3.2.0", "3.3.1", "3.2.2", "3.2.3", "3.2.4"
       ,"3.1.0","3.1.1","3.1.2","3.1.3","3.1.4"
       ,"3.0.0","3.0.1","3.0.2","3.0.3"
    ]


neededColumnsClass =['cbo','cboModified','fanin','fanout','wmc','dit','noc','lcom',
                     'lcom*','tcc','lcc', 'loc'
               ]

base = 'DataSource/'

classCkMetricsAllVersionsDfClass = pd.read_csv(base + 'hadoop_AllCkMetricsAllVersionsClass.csv')
hadoopBugsAllDf = pd.read_csv(base + 'hadoop_bugs_all.csv')
hadoopNewFeaturesAllDf = pd.read_csv(base + 'hadoop_newFeature_all.csv')
hadoopImprovementsAllDf = pd.read_csv(base + 'hadoop_improvements_all.csv')

cleanedCkMetricsAllVersionsDf = (classCkMetricsAllVersionsDfClass
    .loc[~classCkMetricsAllVersionsDfClass['class'].str.contains("Test")]
    .loc[~classCkMetricsAllVersionsDfClass['class'].str.contains("test")]
    .loc[~classCkMetricsAllVersionsDfClass['class'].str.contains("TEST")]
    .loc[~classCkMetricsAllVersionsDfClass['file'].str.contains("src/test/")]
) 

print("Filtrelenmiş Sınıf Sayısı:", cleanedCkMetricsAllVersionsDf.shape[0])
print("Versionlanmış hata sayısı:", hadoopBugsAllDf.shape[0])
print("Versionlanmış yeni özellik sayısı:", hadoopImprovementsAllDf.shape[0])
print("Versionlanmış geliştirme sayısı:", hadoopNewFeaturesAllDf.shape[0])

hadoopBugsGroupedDf = hadoopBugsAllDf.groupby(['AVersion']).count().reset_index()
hadoopImprovementsGroupedDf= hadoopImprovementsAllDf.groupby(['Fix Version/s']).count().reset_index()
hadoopNewFeaturesGroupedDf= hadoopNewFeaturesAllDf.groupby(['Fix Version/s']).count().reset_index()

hadoopBugsGroupedDf = hadoopBugsGroupedDf.rename(columns = {'AVersion': 'Version','Issue key': 'Count'})[['Version','Count']]
hadoopImprovementsGroupedDf = hadoopImprovementsGroupedDf.rename(columns = {'Fix Version/s': 'Version','Issue key': 'Count'})[['Version','Count']]
hadoopNewFeaturesGroupedDf = hadoopNewFeaturesGroupedDf.rename(columns = {'Fix Version/s': 'Version','Issue key': 'Count'})[['Version','Count']]


df = pd.merge(cleanedCkMetricsAllVersionsDf, hadoopBugsGroupedDf,on="Version", how='inner')
df = df[['cbo','cboModified','fanin','fanout','wmc','dit','noc','lcom',
                     'lcom*','tcc','lcc', 'loc', 'Version', 'Count'
               ]]

df['numericVersion'] = df['Version'].apply(lambda x: int(''.join(x.split('.'))))
df = df.drop(columns=['Version'])


df.info()
df.columns = df.columns.str.upper()

std_devs = df.std()

# Standart sapmalara göre sıralayalım ve en düşük 5 standart sapmayı seçelim
lowest_std_devs = std_devs.nsmallest(8)

# En düşük 5 standart sapmaya sahip değişkenlerin isimlerini alalım
selected_columns_std = lowest_std_devs.index.tolist()
#selected_columns = selected_columns_std
# selected_columns.append('COUNT')
# Yeni DataFrame'i oluşturalım
#df = df[selected_columns]
df = df.dropna(axis=0)

# Aykırı değerleri çıkarma fonksiyonu
def remove_outliers(df, columns):
    for col in columns:
        z_scores = stats.zscore(df[col])
        threshold = 3
        df = df[(z_scores < threshold) & (z_scores > -threshold)]
    return df

# Aykırı değerleri çıkar
outliers_removed_df = remove_outliers(df, ['LCOM*', 'TCC', 'LCC', 'NOC', 'DIT', 'FANIN', 'CBO', 'FANOUT'])

df = outliers_removed_df

print(selected_columns_std)

len(df)


Filtrelenmiş Sınıf Sayısı: 445474
Versionlanmış hata sayısı: 21640
Versionlanmış yeni özellik sayısı: 3139
Versionlanmış geliştirme sayısı: 258
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445474 entries, 0 to 445473
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   cbo             445474 non-null  int64  
 1   cboModified     445474 non-null  int64  
 2   fanin           445474 non-null  int64  
 3   fanout          445474 non-null  int64  
 4   wmc             445474 non-null  int64  
 5   dit             445474 non-null  int64  
 6   noc             445474 non-null  int64  
 7   lcom            445474 non-null  int64  
 8   lcom*           440313 non-null  float64
 9   tcc             344210 non-null  float64
 10  lcc             344210 non-null  float64
 11  loc             445474 non-null  int64  
 12  Count           445474 non-null  int64  
 13  numericVersion  445474 non-null  int64  
dtypes: f

315531

In [10]:


rows, columns = df.shape
print(f"Satır sayısı: {rows}, Sütun sayısı: {columns}")

lr= LinearRegression()

X = df[['CBO', 'CBOMODIFIED', 'FANIN', 'FANOUT', 'WMC', 'DIT', 'NOC', 'LCOM',
       'LCOM*', 'TCC', 'LCC', 'LOC']]
y = df['COUNT']



print(X.shape)
print(y.shape)

Y = y.values.reshape(-1, 1)

x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.30, random_state=100)
print("X_train boyutu", x_train.shape)
print("x_test boyutu", x_test.shape)
print("y_train boyutu", y_train.shape)
print("y_test boyutu", y_test.shape)

lr.fit(x_train, y_train)

katsayılar = pd.DataFrame(lr.coef_, columns = [['CBO', 'CBOMODIFIED', 'FANIN', 'FANOUT', 'WMC', 'DIT', 'NOC', 'LCOM',
       'LCOM*', 'TCC', 'LCC', 'LOC']])

katsayılar

y_pred = lr.predict(x_test)




# Means Squared Error Hesaplama

r_2 = r2_score(y_test, y_pred)

print(r_2 *100)


Satır sayısı: 315531, Sütun sayısı: 14
(315531, 12)
(315531,)
X_train boyutu (220871, 12)
x_test boyutu (94660, 12)
y_train boyutu (220871, 1)
y_test boyutu (94660, 1)
-0.009392258356544225


In [11]:
import statsmodels.api as sm


X_train_ols = sm.add_constant(x_train)

sm_model = sm.OLS(y_train, X_train_ols)

sonuc = sm_model.fit()

print(sonuc.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                  0.001893
Date:                Fri, 26 Jan 2024   Prob (F-statistic):               1.00
Time:                        09:54:52   Log-Likelihood:            -1.4875e+06
No. Observations:              220871   AIC:                         2.975e+06
Df Residuals:                  220859   BIC:                         2.975e+06
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         125.4468      1.130    111.060      